# Post Regrid Data Handling

Goal is to have a streamlined way to access the files a user needs given a set of parameters. Envisioning something where a user could enter:
* Sectors of interest (or "all")
* Timerange of interest
* Spatial bounding box
* Gas species of interest    

And be returned either an xarray object with dask parallelization, or create a new netCDF/set of netCDF files to be loaded by the user later. 

There is obviously lots of work to be done, but this is a good start if you want to start playing with data. 


In [1]:
import xarray as xr
import pandas as pd
import os
import pyproj
import numpy as np
import xesmf as xe
import calendar
import datetime
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.img_tiles as cimgt
import matplotlib.pyplot as plt
import sys
sys.path.append('..')

import noaa_csl_funcs as ncf

In [2]:
map_extent={'lon_min':-112.4,
            'lon_max':-111.4,
            'lat_min':40.1,
            'lat_max':41.3} 


# Load from regridded

In [3]:
regrid_id = '_0.025deg'
regridded_path = f'/uufs/chpc.utah.edu/common/home/lin-group9/agm/NOAA_CSL_Data/regridded{regrid_id}'
RCH = ncf.Regridded_CSL_Handler(regridded_path)
dt1  = pd.to_datetime('2019-01-01 00') 
dt2 = pd.to_datetime('2019-02-28 23') 
day_types = ['weekdy','satdy','sundy'] #a list with any or all of 'weekdy','satdy','sundy'
species = ['CO2','CO','HC01','HC02','HC14','NH3','NOX','SO2']
dataset_extent = {'lon_min':-112.1,
                  'lon_max':-111.6,
                  'lat_min':40.3,
                  'lat_max':41.1} 

In [4]:
sector_types = ['area','point']
combined_dss = {}
for sector_type in sector_types:
    #Get the paths to the files that match the criteria
    days_paths = RCH.get_days_in_range(dt1,dt2,day_types,sector_type) 
    files = RCH.get_files_in_days(days_paths)

    load_vars = [] #the variables we want to load
    if sector_type == 'area': #for area sources, just load the species defined above
        load_vars = species.copy() 
    if sector_type == 'point': #for point, often useful to have the stack/type information 
        load_vars = species.copy()
        load_vars.extend(['ITYPE','STKht','STKdiam','STKtemp','STKve','STKflw','FUGht']) #so add it to the actual species

    #Load the files with xarray, preprocessing them so they can be combined by coordinates
    ds_list = [] #initialize the list of datasets
    for file in files:
        ds = RCH.preprocess_regridded(xr.open_dataset(file,chunks = {'utc_hour':1}),dataset_extent)[load_vars] #prepreprocess the file, open with dask chunking, and only keep the species of interest
        ds_list.append(ds)  
    ds_combined = xr.combine_by_coords(ds_list,combine_attrs='drop_conflicts') #this is the combined dataset!
    combined_dss[sector_type] = ds_combined

In [5]:
def rework_datetime(ds):
    combined_ds_list = []
    for yr_mo in ds.yr_mo.values:
        year, month = map(int,yr_mo.split('-'))
        month_ds = ds.sel(yr_mo=yr_mo)

        dates_in_month = pd.date_range(start=f'{year}-{month:02}-01', end=f'{year}-{month:02}-{calendar.monthrange(year, month)[1]}')
        dates_by_day_type = {
            'weekdy':[date for date in dates_in_month if date.weekday() < 5],
            'satdy':[date for date in dates_in_month if date.weekday() == 5],
            'sundy':[date for date in dates_in_month if date.weekday() == 6]
        }

        new_month_ds_list = []
        for day_type,dates in dates_by_day_type.items():
            subds = month_ds.sel(day_type=day_type).drop_vars(['day_type','yr_mo'])
            subds = subds.assign_coords({'date':dates})
            datetimes = [pd.Timestamp(date) + pd.Timedelta(hours=int(hour)) for date in subds.coords['date'].values for hour in subds.coords['utc_hour'].values]
            datetime_index = pd.DatetimeIndex(datetimes)

            subds = subds.stack({'datetime':('date','utc_hour')})#.assign_coords({'datetime':datetime_index})
            subds = subds.drop_vars(['date','utc_hour','datetime'])
            subds = subds.assign_coords({'datetime':datetime_index})
            new_month_ds_list.append(subds)

        new_month_ds = xr.concat(new_month_ds_list,dim='datetime').sortby('datetime')
        combined_ds_list.append(new_month_ds)

    combined_ds = xr.concat(combined_ds_list,dim='datetime').sortby('datetime')
    return combined_ds
    


In [6]:
dt_dss = {}
for sector_type in sector_types:
    dt_dss[sector_type] = rework_datetime(combined_dss[sector_type]).load()


In [7]:
#Save to nc
save_path = '/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/NC'
save_prefix = 'slv_2019_dt'
for sector_type in sector_types:
    dt_dss[sector_type].to_netcdf(os.path.join(save_path,f"{save_prefix}_{sector_type}.nc"))

# Load from presaved

In [3]:
regrid_id = 2
regridded_path = f'/uufs/chpc.utah.edu/common/home/lin-group9/agm/NOAA_CSL_Data/regridded{regrid_id}'
RCH = ncf.Regridded_CSL_Handler(regridded_path)
day_types = ['weekdy','satdy','sundy'] #a list with any or all of 'weekdy','satdy','sundy'

save_path = '/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/NC'
save_prefix = f'slc_2019_em27_{regrid_id}'

#Load from nc
sector_types = ['area','point']
combined_dss = {}
for sector_type in sector_types:
    combined_dss[sector_type] = xr.load_dataset(os.path.join(save_path,f"{save_prefix}_{sector_type}.nc"))

# Analyze

In [ ]:
proj = ccrs.PlateCarree()
fig,axs = plt.subplots(1,2, figsize = (24,10), subplot_kw={'projection':proj})

request = cimgt.GoogleTiles(style='satellite')
scale = 9.0

for i in range(0,2):
    axs[i].set_extent([map_extent['lon_min'],map_extent['lon_max'],map_extent['lat_min'],map_extent['lat_max']],crs=proj)
    axs[i].add_image(request,int(scale))

sector = 'area_onroad_gasoline'
var = 'CO2'
year = 2019
month = 1
day = 1
day_type = 'weekdy'
hour = 11


orig_plot_ds = area_ds.sel(sector=sector,day_type = day_type, yr_mo = f'{year}-{month}',utc_hour = hour)[var]
orig_plot = orig_plot_ds.plot.pcolormesh('lon','lat',ax=axs[0],alpha = 0.5,transform=proj,add_colorbar=True)

combined_plot_ds = combined_ds.sel(sector=sector,datetime=pd.Timestamp(f'{year}-{month}-{day} 00:00:00')+pd.Timedelta(hours=hour))[var]
combined_plot = combined_plot_ds.plot.pcolormesh('lon','lat',ax=axs[1],alpha = 0.5,transform=proj,add_colorbar=True)


## Older stuff

In [4]:
area_ds = combined_dss['area']
day_sum_ds = xr.combine_by_coords([
    area_ds[spec].sum(dim='utc_hour').assign_attrs(
        {'units': f"{area_ds[spec].attrs['units'].split()[0]} day^-1 meters^-2"}
    ) 
    for spec in area_ds.data_vars
    ])

month_sums = []
for yr_mo in day_sum_ds.yr_mo.values:
    yr = int(yr_mo.split('-')[0]) #get the year int
    mo = int(yr_mo.split('-')[1]) #get the mnoth int
    sat,sun,wkdy = ncf.ncount_satsunwkd(yr,mo) #for that year and month, get the number of saturdays, sundays, and weekdays
    sat_sum = (day_sum_ds.sel(yr_mo=yr_mo,day_type='satdy')*sat).reset_coords('day_type',drop=True)#.assign_attrs({'units':f'{mass_unit} month^-1 meters^-2'})
    sun_sum = (day_sum_ds.sel(yr_mo=yr_mo,day_type='sundy')*sun).reset_coords('day_type',drop=True)#.assign_attrs({'units':f'{mass_unit} month^-1 meters^-2'})
    wkdy_sum = (day_sum_ds.sel(yr_mo=yr_mo,day_type='weekdy')*wkdy).reset_coords('day_type',drop=True)#.assign_attrs({'units':f'{mass_unit} month^-1 meters^-2'})
    #wkdy_sum
    month_sum = sat_sum + sun_sum + wkdy_sum
    for var in day_sum_ds.data_vars:
        mass_unit = day_sum_ds[var].attrs['units'].split()[0]
        month_sum[var].attrs.update({'units':f'{mass_unit} month^-1 m^-2'})
    month_sums.append(month_sum)#.reset_coords('yr_mo',drop=True))

yr_sum = xr.concat(month_sums,dim='yr_mo').sum(dim='yr_mo')
for var in yr_sum.data_vars:
    mass_unit = month_sums[0][var].attrs['units'].split()[0]
    yr_sum[var].attrs.update({'units':f'{mass_unit} yr^-1 m^-2'})

In [5]:
grid_area = xr.open_dataset(f'../regridding/grid_area/grid_out_area{regrid_id}.nc')  #load the gridcell area file
grid_area = ncf.slice_extent(grid_area,dataset_extent) #slice it to the same extent
absolute_emissions = (yr_sum * grid_area['cell_area'])#.assign_attrs({'units':mass_unit}) #multiply by the yearsum

for var in yr_sum.data_vars:
    mass_unit = yr_sum[var].attrs['units'].split()[0]
    absolute_emissions[var].attrs.update({'units':f'{mass_unit} yr^-1 gridcell-1'})

absolute_emissions['HC01'] = (absolute_emissions['HC01']*16.04/1E6).assign_attrs({'units':'metric_Ton yr^-1 gridcell^-1'})

In [6]:
species = ['HC01','CO2','CO']

point_ds = combined_dss['point']
day_sum_ds = xr.combine_by_coords([
    point_ds[spec].sum(dim='utc_hour').assign_attrs(
        {'units': f"{point_ds[spec].attrs['units'].split()[0]} day^-1"}
    ) 
    for spec in species
    ])

month_sums = []
for yr_mo in day_sum_ds.yr_mo.values:
    yr = int(yr_mo.split('-')[0]) #get the year int
    mo = int(yr_mo.split('-')[1]) #get the mnoth int
    sat,sun,wkdy = ncf.ncount_satsunwkd(yr,mo) #for that year and month, get the number of saturdays, sundays, and weekdays
    sat_sum = (day_sum_ds.sel(yr_mo=yr_mo,day_type='satdy')*sat).reset_coords('day_type',drop=True)#.assign_attrs({'units':f'{mass_unit} month^-1 meters^-2'})
    sun_sum = (day_sum_ds.sel(yr_mo=yr_mo,day_type='sundy')*sun).reset_coords('day_type',drop=True)#.assign_attrs({'units':f'{mass_unit} month^-1 meters^-2'})
    wkdy_sum = (day_sum_ds.sel(yr_mo=yr_mo,day_type='weekdy')*wkdy).reset_coords('day_type',drop=True)#.assign_attrs({'units':f'{mass_unit} month^-1 meters^-2'})
    #wkdy_sum
    month_sum = sat_sum + sun_sum + wkdy_sum
    for var in day_sum_ds.data_vars:
        mass_unit = day_sum_ds[var].attrs['units'].split()[0]
        month_sum[var].attrs.update({'units':f'{mass_unit} month^-1'})
    month_sums.append(month_sum)#.reset_coords('yr_mo',drop=True))

yr_sum = xr.concat(month_sums,dim='yr_mo').sum(dim='yr_mo')
for var in yr_sum.data_vars:
    mass_unit = month_sums[0][var].attrs['units'].split()[0]
    yr_sum[var].attrs.update({'units':f'{mass_unit} year^-1'})

yr_sum['HC01'] = (yr_sum['HC01']*16.04/1E6).assign_attrs({'units':'metric_Ton yr^-1'})

point_df = yr_sum['HC01'].to_dataframe()
point_df = point_df.reset_index().drop(columns=['ROW'])

In [7]:
landfills = {'slv_landfill':[40.746,-112.042],
             'jordan_landfill':[40.55862,-112.053],
             'davis_landfill':[41.114,-111.931],
             'weber_landfill':[41.218,-111.99],
             'bountiful_landfill':    [40.911,-111.917]}
ww_plants = {'central_valley_wwtp':[40.7036613,-111.9141398],
             'big_cottonwood_wwtp':[40.6187424,-111.7824328],
             'se_regional_wwtp':[40.5411975,-111.8191652],
             'south_valley_wwtp':[40.5033357,-111.9187493],
             'slc_wwtp':[40.8030915,-111.9295899],
             }
refineries = {'Chevron':        [40.825,-111.924],
              'Big West Oil':   [40.838,-111.920],
              'Marathon':       [40.794,-111.909],
              'Holly Refining': [40.887,-111.904],
              'Silver Eagle':   [40.868,-111.910]}

point_sources = pd.concat([pd.DataFrame(landfills),pd.DataFrame(ww_plants),pd.DataFrame(refineries)],axis=1).T
point_sources.columns = ['lat','lon']
point_sources['type'] = point_sources.apply(lambda x: 'landfill' if x.name in landfills.keys() else 'wwtp' if x.name in ww_plants.keys() else 'refinery',axis=1)

In [ ]:
map_extent={'lon_min':-112.2,
        'lon_max':-111.55,
        'lat_min':40.3,
        'lat_max':41.1} 

dataset_extent = {'lon_min':-112.16,
                  'lon_max':-111.7,
                  'lat_min':40.4,
                  'lat_max':41.0} 

var = 'HC01'

plotds = ncf.slice_extent(absolute_emissions[var].sum(dim = 'sector'),dataset_extent).assign_attrs({'units':'metric_Ton yr^-1 gridcell^-1'})
print(plotds.sum())

labsize = 12
proj = ccrs.PlateCarree()
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection = proj)
ax.set_extent([map_extent['lon_min'],map_extent['lon_max'],map_extent['lat_min'],map_extent['lat_max']],crs=proj)

request = cimgt.GoogleTiles(style='street')
scale = 10.0 # prob have to adjust this
ax.add_image(request,int(scale))

plotds.plot.pcolormesh('lon','lat',ax = ax,cmap = 'viridis',alpha = 0.7)

ax.scatter(point_sources.loc[point_sources['type']=='landfill']['lon'],point_sources.loc[point_sources['type']=='landfill']['lat'],c = 'red',s = 100,transform = proj,label = 'landfill')
ax.scatter(point_sources.loc[point_sources['type']=='wwtp']['lon'],point_sources.loc[point_sources['type']=='wwtp']['lat'],c = 'blue',s = 100,transform = proj,label = 'wwtp')
ax.scatter(point_sources.loc[point_sources['type']=='refinery']['lon'],point_sources.loc[point_sources['type']=='refinery']['lat'],c = 'green',s = 100,transform = proj,label = 'refinery')

ax.legend()
ax.set_title(f'All Sectors Summed',fontsize = 10)
plt.show()

In [ ]:
map_extent={'lon_min':-112.2,
        'lon_max':-111.55,
        'lat_min':40.3,
        'lat_max':41.1} 

dataset_extent = {'lon_min':-112.16,
                  'lon_max':-111.7,
                  'lat_min':40.4,
                  'lat_max':41.0} 


for area_sector in absolute_emissions['sector']:
        sector_tag = str(area_sector.values).split('_')[1]

        plotds = ncf.slice_extent(absolute_emissions['HC01'].sel(sector=area_sector),dataset_extent)
        point_plot_df = point_df[point_df['sector'] == f'point_{sector_tag}']

        labsize = 12
        proj = ccrs.PlateCarree()
        fig = plt.figure(figsize=(10,10))
        ax = plt.axes(projection = proj)
        ax.set_extent([map_extent['lon_min'],map_extent['lon_max'],map_extent['lat_min'],map_extent['lat_max']],crs=proj)

        request = cimgt.GoogleTiles(style='street')
        scale = 10.0 # prob have to adjust this
        ax.add_image(request,int(scale))



        plotds.plot.pcolormesh('lon','lat',ax = ax,cmap = 'viridis',alpha = 0.7)
        #ax.scatter(point_plot_df['lon'],point_plot_df['lat'],c = point_plot_df['HC01'],cmap = 'Reds',transform = proj)
        ax.coastlines()
        ax.add_feature(cfeature.BORDERS)
        ax.add_feature(cfeature.STATES)

        ax.scatter(point_sources.loc[point_sources['type']=='landfill']['lon'],point_sources.loc[point_sources['type']=='landfill']['lat'],c = 'red',s = 100,transform = proj,label = 'landfill')
        ax.scatter(point_sources.loc[point_sources['type']=='wwtp']['lon'],point_sources.loc[point_sources['type']=='wwtp']['lat'],c = 'blue',s = 100,transform = proj,label = 'wwtp')
        ax.scatter(point_sources.loc[point_sources['type']=='refinery']['lon'],point_sources.loc[point_sources['type']=='refinery']['lat'],c = 'green',s = 100,transform = proj,label = 'refinery')

                
        ax.legend()
        plt.show()

In [ ]:
combined_dss['area']

In [ ]:
plotds = combined_dss['area'].sel({'sector':'area_onroad_gasoline','utc_hour':0,'day_type':'weekdy','yr_mo':'2019-1'})['CO2']

map_extent={'lon_min':-112.2,
        'lon_max':-111.55,
        'lat_min':40.3,
        'lat_max':41.1} 

dataset_extent = {'lon_min':-112.16,
                  'lon_max':-111.7,
                  'lat_min':40.4,
                  'lat_max':41.0} 



labsize = 12
proj = ccrs.PlateCarree()
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection = proj)
ax.set_extent([map_extent['lon_min'],map_extent['lon_max'],map_extent['lat_min'],map_extent['lat_max']],crs=proj)

request = cimgt.GoogleTiles(style='street')
scale = 10.0 # prob have to adjust this
ax.add_image(request,int(scale))

plotds.plot.pcolormesh('lon','lat',ax = ax,cmap = 'viridis',alpha = 0.7)


ax.legend()
ax.set_title(f'All Sectors Summed',fontsize = 10)
plt.show()